# Implementação Modelo Alocação Blitz

Aqui apresentamos um modelo generico para o problema de Alocação de Blitz. Neste problema temos um conjunto de pontos válidos para alocar blitz e um conjunto de equipes. Dados essas entradas desejamos maximizar um certa função objetivo atendendo as restrições das equipes.

## Definindo paramêtros de entrada

Fixando dimensões iniciais.

In [13]:
n_pontos = 20
n_equipes = 4
tempo_total = 48


48

Gerando $\mu_{it}$ aleatórios para função objetivo.

In [14]:
using Random
using Distributions
using LinearAlgebra
rng = MersenneTwister(1234)
Mu = rand(50:1000,n_pontos,tempo_total)
print(Mu)

[70 494 650 610 494 102 844 964 96 406 88 611 740 986 857 169 317 864 106 813 149 548 462 990 977 943 394 287 697 842 884 308 331 187 608 151 68 646 931 188 784 320 984 87 790 281 872 472; 551 998 397 853 303 933 581 400 397 750 603 892 256 757 886 646 503 886 481 865 892 288 866 314 882 290 270 965 855 947 353 387 894 813 452 673 925 711 494 641 240 775 476 869 287 887 805 622; 432 385 223 648 235 228 278 392 832 988 147 821 682 225 849 663 994 498 785 512 921 893 360 322 190 330 934 812 80 599 441 808 300 380 475 439 167 118 56 451 616 202 382 862 192 208 77 572; 951 824 434 531 665 720 813 612 876 220 145 379 683 221 566 373 592 974 326 510 388 808 111 300 223 870 53 164 310 484 561 73 65 610 447 539 183 342 175 854 610 573 603 761 141 802 963 254; 594 612 130 276 112 684 144 979 641 647 140 987 367 390 276 913 655 629 662 792 858 120 647 313 409 589 970 974 835 362 836 974 379 328 551 781 349 478 601 957 820 591 776 195 563 756 141 430; 655 262 663 490 85 190 633 966 122 205 201 18

Gerando uma mapa de disponibilidade para equipes em forma de bitmap o qual é possível estender para diferentes restrições. Caso 1 significa que a $k$-esima($k \in K$) equipe está disponível no ponto $i\in I$ no tempo $t\in T$. Caso contrario existe uma restrição proibindo está alocação.

In [15]:
MapaViabilidade = reshape(bitrand(n_equipes*n_pontos*tempo_total),n_equipes,n_pontos,tempo_total)

4×20×48 BitArray{3}:
[:, :, 1] =
 1  0  1  0  1  1  1  1  1  0  0  1  1  1  0  1  0  1  1  1
 0  1  0  1  1  0  1  1  1  0  0  0  0  1  0  1  1  0  1  1
 0  0  1  1  1  1  0  1  1  1  0  0  0  0  0  1  1  1  1  1
 1  1  1  0  0  0  1  0  1  1  0  0  1  0  1  0  1  1  1  0

[:, :, 2] =
 1  1  0  0  1  1  0  1  1  0  1  1  0  1  0  0  0  0  1  0
 1  1  0  1  0  1  0  1  1  1  0  0  0  1  0  1  1  1  0  1
 1  1  0  0  0  0  0  0  0  0  0  1  1  1  0  0  1  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  1  0  1  0  0  1  1  1  0

[:, :, 3] =
 1  1  1  0  0  1  1  1  1  0  1  0  1  0  1  1  1  0  0  1
 1  1  1  0  1  0  1  1  1  1  1  1  1  0  1  1  1  1  0  0
 1  0  0  0  1  1  0  1  1  0  1  0  1  1  0  0  0  1  0  1
 0  1  1  0  1  0  0  1  1  0  0  0  1  1  1  1  1  0  0  0

...

[:, :, 46] =
 0  0  1  0  1  0  1  0  1  0  1  1  0  0  1  0  1  1  0  0
 0  1  1  0  0  1  1  1  1  0  1  1  1  1  1  1  1  0  0  1
 0  1  1  1  0  0  1  0  1  0  1  1  1  1  1  1  0  1  1  0
 0  0  1  0  0  0  1  

## Inicializando Solver

Usando para prototipação o solver GLPK inicialmente, provavelmente será modificado para testes.

In [16]:
using JuMP
using CPLEX
using MathOptInterface
#setting const
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities
#initiallising solver model
model = Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

## Variáveis de decisão

 
 
-  $x_{it}\in \{0,1\}$ será uma variável usada para representar com 1 a presença de uma blitz na localização $i\in I$ no período de tempo $t \in T$, e com 0 caso contrário;

In [17]:
@variable(model,x[i=1:n_pontos,t = 1:tempo_total],Bin)

20×48 Array{VariableRef,2}:
 x[1,1]   x[1,2]   x[1,3]   x[1,4]   x[1,5]   …  x[1,46]   x[1,47]   x[1,48]
 x[2,1]   x[2,2]   x[2,3]   x[2,4]   x[2,5]      x[2,46]   x[2,47]   x[2,48]
 x[3,1]   x[3,2]   x[3,3]   x[3,4]   x[3,5]      x[3,46]   x[3,47]   x[3,48]
 x[4,1]   x[4,2]   x[4,3]   x[4,4]   x[4,5]      x[4,46]   x[4,47]   x[4,48]
 x[5,1]   x[5,2]   x[5,3]   x[5,4]   x[5,5]      x[5,46]   x[5,47]   x[5,48]
 x[6,1]   x[6,2]   x[6,3]   x[6,4]   x[6,5]   …  x[6,46]   x[6,47]   x[6,48]
 x[7,1]   x[7,2]   x[7,3]   x[7,4]   x[7,5]      x[7,46]   x[7,47]   x[7,48]
 x[8,1]   x[8,2]   x[8,3]   x[8,4]   x[8,5]      x[8,46]   x[8,47]   x[8,48]
 x[9,1]   x[9,2]   x[9,3]   x[9,4]   x[9,5]      x[9,46]   x[9,47]   x[9,48]
 x[10,1]  x[10,2]  x[10,3]  x[10,4]  x[10,5]     x[10,46]  x[10,47]  x[10,48]
 x[11,1]  x[11,2]  x[11,3]  x[11,4]  x[11,5]  …  x[11,46]  x[11,47]  x[11,48]
 x[12,1]  x[12,2]  x[12,3]  x[12,4]  x[12,5]     x[12,46]  x[12,47]  x[12,48]
 x[13,1]  x[13,2]  x[13,3]  x[13,4]  x[13,5] 

-      $e_{ijt}\in \{0,1\}$ será uma variável usada para representar com 1 a presença de uma equipe  $k \in K$ na localização $i\in I$ no período de tempo $t \in T$, e com 0 caso contrário;

In [18]:
@variable(model,e[k=1:n_equipes,i=1:n_pontos,t = 1:tempo_total],Bin)

4×20×48 Array{VariableRef,3}:
[:, :, 1] =
 e[1,1,1]  e[1,2,1]  e[1,3,1]  e[1,4,1]  …  e[1,18,1]  e[1,19,1]  e[1,20,1]
 e[2,1,1]  e[2,2,1]  e[2,3,1]  e[2,4,1]     e[2,18,1]  e[2,19,1]  e[2,20,1]
 e[3,1,1]  e[3,2,1]  e[3,3,1]  e[3,4,1]     e[3,18,1]  e[3,19,1]  e[3,20,1]
 e[4,1,1]  e[4,2,1]  e[4,3,1]  e[4,4,1]     e[4,18,1]  e[4,19,1]  e[4,20,1]

[:, :, 2] =
 e[1,1,2]  e[1,2,2]  e[1,3,2]  e[1,4,2]  …  e[1,18,2]  e[1,19,2]  e[1,20,2]
 e[2,1,2]  e[2,2,2]  e[2,3,2]  e[2,4,2]     e[2,18,2]  e[2,19,2]  e[2,20,2]
 e[3,1,2]  e[3,2,2]  e[3,3,2]  e[3,4,2]     e[3,18,2]  e[3,19,2]  e[3,20,2]
 e[4,1,2]  e[4,2,2]  e[4,3,2]  e[4,4,2]     e[4,18,2]  e[4,19,2]  e[4,20,2]

[:, :, 3] =
 e[1,1,3]  e[1,2,3]  e[1,3,3]  e[1,4,3]  …  e[1,18,3]  e[1,19,3]  e[1,20,3]
 e[2,1,3]  e[2,2,3]  e[2,3,3]  e[2,4,3]     e[2,18,3]  e[2,19,3]  e[2,20,3]
 e[3,1,3]  e[3,2,3]  e[3,3,3]  e[3,4,3]     e[3,18,3]  e[3,19,3]  e[3,20,3]
 e[4,1,3]  e[4,2,3]  e[4,3,3]  e[4,4,3]     e[4,18,3]  e[4,19,3]  e[4,20,3]

...

[:, :, 46] =
 

## Restrições Iniciais

In [19]:
@constraint(model,first_constraint[i=1:n_pontos,t=1:tempo_total],x[i,t] == sum(e[k,i,t] for k in 1:n_equipes))
@constraint(model,second_constraint[k=1:n_equipes,t=1:tempo_total],sum(e[k,i,t] for i in 1:n_pontos)<=1)
@constraint(model,third_constraint[k=1:n_equipes,i=1:n_pontos,t=2:tempo_total-1],e[k,i,t-1]+sum(e[k,j,t] for j=1:n_pontos if j !=i)<=1)
@constraint(model,fourth_constraint,sum(e[k,i,t]*(1-MapaViabilidade[k,i,t]) for t in 1:tempo_total,i in 1:n_pontos,k in 1:n_equipes)<=0)
obj = dot(Mu,x)
@objective(model,Max,obj)
optimize!(model)



Clique cuts applied:  98
Zero-half cuts applied:  93
Lift and project cuts applied:  11
Gomory fractional cuts applied:  28

Root node processing (before b&c):
  Real time             =    0.76 sec. (571.05 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.76 sec. (571.05 ticks)
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 114208.0000.
Tried aggregator 2 times.
MIP Presolve eliminated 2762 rows and 1957 columns.
MIP Presolve modified 1159 coefficients.
Aggregator did 230 substitutions.
Reduced MIP has 1841 rows, 2613 columns, and 19913 nonzeros.
Reduced MIP has 2613 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (30.93 ticks)
Probing time = 0.02 sec. (1.40 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 16 rows and 47 column

In [20]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [21]:
optimal_solution = value.(x)

20×48 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [22]:
optimal_team_schedule = value.(e)

4×20×48 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0
